### Model Creation - Multi Layer Perceptron
Paper:  
Sigmoid activation function has been considered in this work
as the non-linear transfer function of each neuron. We utilized the
Scikit-Learn library [32] in Python to create the model

**Imports:**

In [429]:
import torch
import csv
from torch.utils.data import random_split
from matplotlib import pyplot
import pandas as pd
import torch



#### Dataset Loading

In [430]:
def prepare_dataset(dataset_files, delimiter=";"):
  
    dataframes = []
    data_path = "../data/"
    
    for file in dataset_files:
      df = pd.read_csv(data_path+file, delimiter=delimiter)
              
      label = 0
      if "atk" in file:
        # df.drop(df.columns[0], axis=1, inplace=True) # drop index-column
        label = 1
      df["label"] = label
            
      dataframes.append(df)
        
    concatenated_df = pd.concat(dataframes, axis=0)
    concatenated_df.replace({True: 1, False: 0}, inplace=True) # convert boolean to numbers    
    concatenated_df.drop(columns=["blockNumber", "address", "transactionHash"], inplace=True)

    # Convert dataframe to torch tensor
    data_tensor = torch.tensor(concatenated_df.values, dtype=torch.float)
    
    # Get the input (data samples) without the target information
    X = data_tensor[:, :-1]
    
    # Get the target information
    T = data_tensor[:, -1].unsqueeze(1)

    print("Features: ", concatenated_df.columns)

    print(f"Loaded dataset with {len(concatenated_df)} samples")
    
    return X, T


In [431]:


dataset_files = ["insertion_atks_first_atk.csv",
                 "insertion_atks_second_atk.csv",
                 "random_sampled_transactions.csv"]

X, T = prepare_dataset(dataset_files, ",")

#### Split Training and Validation Data

In [432]:
def split_training_data(X, T, train_percentage=0.8):
  
  assert len(X) == len(T)
  
  num_samples = len(X)
  
  train_size = int(train_percentage * num_samples)
  validation_size = num_samples - train_size
    
  # split into 80/20 training/validation
  idx_train, idx_validation = random_split(range(num_samples), [train_size, validation_size])

  
  X_train = X[idx_train]
  T_train = T[idx_train]
  X_val = X[idx_validation]
  T_val = T[idx_validation]

  return X_train, T_train, X_val, T_val

#### Input Data Standardization

In [433]:
def standardize(X_train, X_val):
  # compute statistics
  mean = X_train.mean(dim=0)
  std = X_train.std(dim=0)
  
  torch.save(mean, './mlp-mean.pt')
  torch.save(std, './mlp-std.pt')

  # Check if standard deviation is zero
  zero_std_mask = std == 0
  
  # Standardize both X_train and X_val, excluding columns with zero standard deviation
  X_train = (X_train - mean) / std.masked_fill(zero_std_mask, 1)  # Replace zero std with 1 to avoid division by zero
  X_val = (X_val - mean) / std.masked_fill(zero_std_mask, 1)

  return X_train, X_val

### Network Implementation

**Network Architecture**
- 1 input layer
  - 7 input neurons
- 2 hidden layers
  - 10 hidden neurons
  - 10 hidden neurons
- 1 output layer
  - 1 output neuron    

Using Sigmoid Activation function!

In [434]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(7, 10),  # Input layer to first hidden layer
            torch.nn.Sigmoid(),  # Sigmoid activation function for the first hidden layer
            torch.nn.Linear(10, 10),  # First hidden layer to second hidden layer
            torch.nn.Sigmoid(),  # Sigmoid activation function for the second hidden layer
            torch.nn.Linear(10, 1)
        )

    def forward(self, x):
        return self.layers(x)

**Training Loop**

In [435]:
def accuracy(Z, T):
  # binary classification
  z_sign = (Z >= 0.5).int()
  return torch.mean((z_sign == T).float())

In [436]:
def train(network, loss, X_train, T_train, X_val, T_val, num_epochs=10000, learning_rate=0.1):
  
  optimizer = torch.optim.SGD(
    params=network.parameters(),
    lr=learning_rate,
    momentum=0.9
  )

  # collect loss and accuracy values
  train_loss, train_acc, val_loss, val_acc = [], [], [], []

  for epoch in range(num_epochs):
    # train on training set
    optimizer.zero_grad()
    # ... compute network output on training data
    Z = network(X_train)
    
    # ... compute loss from network output and target data
    J = loss(Z, T_train)
    J.backward()
    # ... perform parameter update
    optimizer.step()
    # ... remember loss
    train_loss.append(J.item())
    # ... compute training set accuracy
    train_acc.append(accuracy(Z, T_train).item())
    
    print(f'Epoch: {epoch}/{num_epochs}')

    # test on validation data
    with torch.no_grad():
      # ... compute network output on validation data
      Z = network(X_val)
      # ... compute loss from network output and target data
      J = loss(Z, T_val)
      # ... remember loss
      val_loss.append(J.item())
      # ... compute validation set accuracy
      val_acc.append(accuracy(Z, T_val).item())

  # return the four lists of losses and accuracies
  return train_loss, train_acc, val_loss, val_acc

In [437]:
def plot(train_loss, train_acc, val_loss, val_acc):
  pyplot.figure(figsize=(10,3))
  ax = pyplot.subplot(121)
  ax.plot(train_loss, "g-", label="Training set loss")
  ax.plot(val_loss, "b-", label="Validation set loss")
  ax.legend()

  ax = pyplot.subplot(122)
  ax.plot(train_acc, "g-", label="Training set accuracy")
  ax.plot(val_acc, "b-", label="Validation set accuracy")
  ax.legend()

In [438]:
# Load dataset
#dataset_files = ["insertion_atks_first_atk_2500.csv", "insertion_atks_second_atk_2500.csv", "random_sampled_transactions.csv"]

X, T = prepare_dataset(dataset_files, ",")

# Split dataset
X_train, T_train, X_val, T_val = split_training_data(X, T, 0.8)
# Standardize input data
X_train, X_val = standardize(X_train, X_val)

# Initiate the network
network = Network()
# Define loss function
loss = torch.nn.BCEWithLogitsLoss(reduction="mean")

# Train network on our data
results = train(network, loss, X_train, T_train, X_val, T_val, num_epochs=10000, learning_rate=0.1)
train_loss, train_acc, val_loss, val_acc = results

# plot the results
plot(train_loss, train_acc, val_loss, val_acc)

In [439]:
torch.save(network.state_dict(), './front-running-attack-model.pth')

In [440]:
X_train.shape

In [441]:
print(val_acc[-1])

#### Comparison to other models

**Random Forest**

In [442]:
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.tree import ExtraTreeClassifier


# Initialize the Random Forest classifier
classifiers = [KNeighborsClassifier(3),
                SVC(kernel="linear", C=0.025, random_state=42),
                SVC(gamma=2, C=1, random_state=42),
                #GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
                DecisionTreeClassifier(max_depth=5, random_state=42),
                RandomForestClassifier(
                    max_depth=5, n_estimators=10, max_features=1, random_state=42),
                AdaBoostClassifier(algorithm="SAMME", random_state=42),
                GaussianNB(),
                QuadraticDiscriminantAnalysis(),
                MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='adam', max_iter=10000),
                ExtraTreeClassifier()]

for classifier in classifiers:
    print(classifier)
    classifier.fit(X_train, T_train)
    predictions = classifier.predict(X_val)
    accuracy = accuracy_score(T_val, predictions)
    f1 = f1_score(T_val.numpy(), predictions, average='weighted')
    print("Accuracy:", accuracy)
    print("F1-Score:", f1)
    print('--------------------------------')